# Run Test Cases

In [ ]:
import os
import time
repeats = 10
cores = 32 # change it to the number of the cores
outputDir="measurements/"
def runTest(cmd, testName, languageName, testcases, repeat, heavyThreadtest):
    if heavyThreadtest == testcases: 
        lightOrHeavy = "heavythread"
    else:
        lightOrHeavy = "lightthread"
    for r in range(repeat):
        outputFileName = outputDir+languageName+"."+testName+"."+lightOrHeavy+"."+str(r)+".txt"
        for t in testcases:
            runCmd0 = "echo num: "+str(t) +" >>" + outputFileName
            os.system(runCmd0)
            runCmd = cmd % t + " 2>> " + outputFileName
            bashCmd = 'GREPDB=\"'+runCmd+'\"; /bin/bash -c "$GREPDB"'
            os.system(bashCmd)
            #print(runCmd)
            #time.sleep(1)
def cleanup():
    files = outputDir + "*.txt"
    cmd  = "rm -f " + files
    os.system(cmd)
    files = outputDir + "*.dat"
    cmd  = "rm -f " + files
    os.system(cmd)
    
cleanup()

In [ ]:
heavyThreadTestCases = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]
lightThreadTestCases = [100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000]

####  Part I  #####

#PQ_Lime
cmd = "time (./bin/pingpong_lime %d 1 1)"
runTest(cmd, "pingpong", "Lime",  heavyThreadTestCases, repeats, heavyThreadTestCases)

#PQ_GO
cmd = "time (./bin/pingpong_go %d)"
runTest(cmd, "pingpong", "Go",  heavyThreadTestCases, repeats, heavyThreadTestCases)

#PQ_Erlang
cmd = "time (erl -noshell -pa bin -s -run pingpong start -s init stop -rounds %d)"
runTest(cmd, "pingpong", "Erlang",  heavyThreadTestCases, repeats, heavyThreadTestCases)

#PQ_Haskell
cmd = "time (./bin/pingpong_haskell %d)"
runTest(cmd, "pingpong", "Haskell",  heavyThreadTestCases, repeats, heavyThreadTestCases)

#PQ_Java
cmd = "time (java -cp bin/ PingPong %d)"
runTest(cmd, "pingpong", "Java",  heavyThreadTestCases, repeats, heavyThreadTestCases)

#PQ_Pthread
cmd = "time (./bin/pingpong_pthread %d)"
runTest(cmd, "pingpong", "Pthread",  heavyThreadTestCases, repeats, heavyThreadTestCases)

#PQ_Rust
cmd = "time (./bin/pingpong_rust %d)"
runTest(cmd, "pingpong", "Rust",  heavyThreadTestCases, repeats, heavyThreadTestCases)

####  Part II  #####

#PQ_Lime
cmd = "time (./bin/pingpong_lime %d 1 1)"
runTest(cmd, "pingpong", "Lime",  lightThreadTestCases, repeats, heavyThreadTestCases)

#PQ_GO
cmd = "time (./bin/pingpong_go %d)"
runTest(cmd, "pingpong", "Go",  lightThreadTestCases, repeats, heavyThreadTestCases)

#PQ_Erlang
cmd = "time (erl -noshell -pa bin -s -run pingpong start -s init stop -rounds %d)"
runTest(cmd, "pingpong", "Erlang",  lightThreadTestCases, repeats, heavyThreadTestCases)

#PQ_Haskell
cmd = "time (./bin/pingpong_haskell %d)"
runTest(cmd, "pingpong", "Haskell",  lightThreadTestCases, repeats, heavyThreadTestCases)

#PQ_Rust
#cmd = "time (./bin/pingpong_rust %d)"
#runTest(cmd, "pingpong", "Rust",  lightThreadTestCases, repeats, heavyThreadTestCases)

In [ ]:
import sys
import re
import glob
import resource

allLanguages = [ "Pthread", "Pthread_mon", "Haskell", "Go", "Erlang", "Java", "Lime", "Rust"]

outputDir="measurements/"
datap = re.compile('num:\\s+([0-9]+).*') #group 1
realp = re.compile('real\\s+([0-9]+)m([0-9]+\\.[0-9]+)s.*') #group 1 and group 2
userp = re.compile('user\\s+([0-9]+)m([0-9]+\\.[0-9]+)s.*')
sysp = re.compile('sys\\s+([0-9]+)m([0-9]+\\.[0-9]+)s.*')

def parse():
    for l in allLanguages:
        results = glob.glob("measurements/Results/"+ outputDir +"/*.txt")
        #print(results)
        for f in results:
            ifp=open(f)
            ofp=open(f+'.dat', 'w+')
            line=ifp.readlines()
            row=[None, None]
            for x in line:
                if row[0] is None:
                    m = datap.match(x)
                    if m is not None:
                        row[0] = m.group(1)
                elif row[1] is None:
                    t = realp.match(x)
                    if t is not None:
                        row[1] = str((int(t.group(1))*60+float(t.group(2)))*1000)
                        #print(row[1])
                        ofp.write(row[0])
                        ofp.write(' ')
                        ofp.write(row[1])
                        ofp.write(' ')
                        ofp.write('\n')
                        row = [None, None]
            ofp.close()

parse()

In [ ]:
%%file gendata.sh
./others/gnuplotme --set x file $1.1.txt.dat 1 --set z avg 2 0 95 $1.*.txt.dat --plot x xaxis --plot z avg_example --outfile $1

In [ ]:
%%sh
chmod +x ./others/gnuplotme

In [ ]:
import os
#testcases = ["PQ", "LOT", "MR"]
testcases = ["parallelthreadring"] #["pingpong", "threadring"]#, "parallelthreadring"]
lightLanguages = [ "Haskell", "Go", "Erlang", "Lime", "Rust"]
#lightLanguages = ["Lime"]
outputDir="../Results/tmp-dir/"

for l in allLanguages:
    for t in testcases:
        cmd  = "bash gendata.sh "+outputDir  +l+"."+t+".heavythread"
        os.system(cmd)
for l in lightLanguages:
    for t in testcases:
        cmd  = "bash gendata.sh "+outputDir +l+"."+t+".lightthread"
        os.system(cmd)

In [1]:
%%writefile PingPong.gp

set terminal png
set output "ex_PingPong.png"
#set terminal postscript eps enhanced color font 'Helvetica,10'
#set output "PingPong.eps"
set multiplot layout 1,2 
set tmargin at screen 0.1
set bmargin at screen 0.80
set lmargin at screen 0.15
set rmargin at screen 0.45
set xlabel "Tokens"
set ylabel "Time (ms)"
set xrange [1000:9000]
set xtics rotate by -75 offset -1,0.5,0
set key left top
#set xtics rotate by 0 offset 0,0,0

plot "../Results/tmp-dir/Lime.pingpong.heavythread.data" using 1:2 title 'Lime' with linespoints linecolor rgb "red" pointtype 26 pointsize 2, \
    "../Results/tmp-dir/Go.pingpong.heavythread.data" using 1:2 title 'Go' with linespoints linecolor rgb "green" pointtype 39 pointsize 2, \
    "../Results/tmp-dir/Erlang.pingpong.heavythread.data" using 1:($2-1140) title 'Erlang' with linespoints linecolor rgb "blue" pointtype 10 pointsize 2, \
    "../Results/tmp-dir/Java.pingpong.heavythread.data" using 1:2 title 'Java' with linespoints linecolor rgb "coral" pointtype 70 pointsize 2, \
    "../Results/tmp-dir/Pthread.pingpong.heavythread.data" using 1:2 title 'Pthread' with linespoints linecolor rgb "#5F9EA0" pointtype 1 pointsize 2, \
    "../Results/tmp-dir/Haskell.pingpong.heavythread.data" using 1:2 title 'Haskell' with linespoints linecolor rgb "brown" pointtype 58 pointsize 2, \
    "../Results/tmp-dir/Rust.pingpong.heavythread.data" using 1:2 title 'Rust' with linespoints linecolor rgb 'gold' pointtype 50 pointsize 2        
set lmargin at screen 0.55
set rmargin at screen 0.95
set xlabel "Tokens"
#set ylabel "Time (ms)" offset 3,20,0
set xtics rotate by -75 offset -1,0.5,0
set xrange [100000:900000]
unset key
#set key left top
unset ylabel
plot "../Results/tmp-dir/Lime.pingpong.lightthread.data" using 1:2 title 'Lime' with linespoints linecolor rgb "red" pointtype 26 pointsize 2, \
     "../Results/tmp-dir/Go.pingpong.lightthread.data" using 1:2 title 'Go' with linespoints linecolor rgb "green" pointtype 39 pointsize 2, \
     "../Results/tmp-dir/Erlang.pingpong.lightthread.data" using 1:($2-1100) title 'Erlang' with linespoints linecolor rgb "blue" pointtype 10 pointsize 2, \
     "../Results/tmp-dir/Haskell.pingpong.lightthread.data" using 1:2 title 'Haskell' with linespoints linecolor rgb "brown" pointtype 58 pointsize 2
     #"../Results/tmp-dir/Rust.pingpong.lightthread.data" using 1:2 title 'Rust' with linespoints linecolor rgb "gold" pointtype 50 pointsize 2 
unset multiplot
unset xtics

Writing PingPong.gp


In [ ]:
%%sh
cat PingPong.gp | gnuplot

In [ ]:
from IPython.display import Image
Image(filename='ex_PingPong.png') 